In [2]:
import time
import re
import mysql.connector
from datetime import datetime
import os

# --- CONFIGURAÇÕES DA CORTEX 360 ---
# Ajuste com a senha real do banco 'cortex360' que você usa no aaPanel
DB_CONFIG = {
    'user': 'cortex360',      
    'password': 'Cortex360Vini', 
    'host': '72.60.244.132',
    'database': 'cortex360',
    'raise_on_warnings': True
}

# Caminho do log do Nginx (Geralmente no aaPanel fica aqui):
LOG_FILE = '/www/wwwlogs/cortex360.com.br.log' 
# Se não achar, tente: '/var/log/nginx/access.log'

# Regex para ler o padrão "Combined" do Nginx
# Captura: IP, Data, Método, URL, Status, Bytes, Referer e User Agent
LOG_PATTERN = re.compile(
    r'(?P<ip>[\d\.]+) - - \[(?P<time>.*?)\] "(?P<method>\w+) (?P<path>.*?) HTTP/.*?" (?P<status>\d+) (?P<bytes>\d+) "(?P<referer>.*?)" "(?P<agent>.*?)"'
)

def parse_line(line):
    match = LOG_PATTERN.match(line)
    if match:
        return match.groupdict()
    return None

def connect_db():
    return mysql.connector.connect(**DB_CONFIG)

def main():
    print(f"🚀 CORTEX Log Shipper (MySQL Edition) iniciado... Monitorando: {LOG_FILE}")
    
    conn = connect_db()
    cursor = conn.cursor()

    # Abre o arquivo e vai para o final (Tail -f style)
    try:
        f = open(LOG_FILE, 'r')
        f.seek(0, os.SEEK_END)
    except FileNotFoundError:
        print(f"❌ Erro: Arquivo de log não encontrado em {LOG_FILE}")
        return

    try:
        while True:
            line = f.readline()
            if not line:
                time.sleep(0.1) # Aguarda novos logs entrarem
                continue

            data = parse_line(line)
            if data:
                try:
                    # Converter data do Nginx (10/Jan/2026:20:30:00 +0000) para MySQL (2026-01-10 20:30:00)
                    # Removemos o fuso horário (+0000) para simplificar, ou ajustamos conforme necessidade
                    dt_str = data['time'].split()[0]
                    dt_obj = datetime.strptime(dt_str, '%d/%b/%Y:%H:%M:%S')
                    mysql_time = dt_obj.strftime('%Y-%m-%d %H:%M:%S')
                    
                    sql = """
                        INSERT INTO access_logs 
                        (log_time, ip_address, method, url_path, status_code, bytes_sent, referer, user_agent)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                    """
                    val = (
                        mysql_time, 
                        data['ip'], 
                        data['method'], 
                        data['path'], 
                        int(data['status']), 
                        int(data['bytes']), 
                        data['referer'], 
                        data['agent']
                    )
                    
                    cursor.execute(sql, val)
                    conn.commit()
                    
                    # print(f"✅ [200] {data['path']}") # Descomente para debugar
                    
                except mysql.connector.Error as err:
                    print(f"⚠️ Erro SQL: {err}")
                    # Se cair a conexão, tenta reconectar
                    if not conn.is_connected():
                        print("Reconectando ao MySQL...")
                        conn = connect_db()
                        cursor = conn.cursor()
                except Exception as e:
                    print(f"❌ Erro de Parse: {e}")

    except KeyboardInterrupt:
        print("\n🛑 Shipper parado manualmente.")
        f.close()
        conn.close()

if __name__ == "__main__":
    main()

🚀 CORTEX Log Shipper (MySQL Edition) iniciado... Monitorando: /www/wwwlogs/cortex360.com.br.log
❌ Erro: Arquivo de log não encontrado em /www/wwwlogs/cortex360.com.br.log
